<a href="https://colab.research.google.com/github/bsvinay9/Assignment0/blob/main/Gamma_correction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Default title text
import cv2
import numpy as np
import matplotlib.pyplot as plt

def gamma_correction(image, gamma):
    inv_gamma = 1.0 / gamma
    table = np.array([(i / 255.0) ** inv_gamma * 255 for i in np.arange(0, 256)]).astype("uint8")
    return cv2.LUT(image, table)
from google.colab.patches import cv2_imshow

In [ ]:
def histogram_equalization(image):
    if len(image.shape) == 2:
        return cv2.equalizeHist(image)
    elif len(image.shape) == 3:
        ycrcb = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
        ycrcb[:, :, 0] = cv2.equalizeHist(ycrcb[:, :, 0])
        return cv2.cvtColor(ycrcb, cv2.COLOR_YCrCb2BGR)


In [ ]:
def reduce_bit_depth(image, bit_depth):
    levels = 2 ** bit_depth
    factor = 256 // levels
    return (image // factor) * factor

def reduce_spatial_resolution(image):
    h, w = image.shape[:2]
    resized = cv2.resize(image, (w // 2, h // 2), interpolation=cv2.INTER_AREA)
    return resized

def apply_lowpass_filter(image, kernel_size):
    return cv2.GaussianBlur(image, (kernel_size, kernel_size), 0)



In [ ]:
def lowpass_frequency_filter(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    dft = np.fft.fft2(gray)
    dft_shift = np.fft.fftshift(dft)
    rows, cols = gray.shape
    mask = np.zeros((rows, cols), np.uint8)
    center_x, center_y = cols // 2, rows // 2
    mask[center_y - 30:center_y + 30, center_x - 30:center_x + 30] = 1
    dft_shift_filtered = dft_shift * mask
    dft_filtered = np.fft.ifftshift(dft_shift_filtered)
    img_filtered = np.fft.ifft2(dft_filtered)
    return np.abs(img_filtered)



In [61]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def gamma_correction(image, gamma):
    inv_gamma = 1.0 / gamma
    table = np.array([(i / 255.0) ** inv_gamma * 255 for i in range(256)]).astype("uint8")
    return cv2.LUT(image, table)

def histogram_equalization(image):
    if len(image.shape) == 2:
        return cv2.equalizeHist(image)
    ycrcb = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    ycrcb[:, :, 0] = cv2.equalizeHist(ycrcb[:, :, 0])
    return cv2.cvtColor(ycrcb, cv2.COLOR_YCrCb2BGR)

def reduce_bit_depth(image, bit_depth):
    shift = 8 - bit_depth
    return (image >> shift) << shift  # More accurate bit truncation

def reduce_spatial_resolution(image):
    h, w = image.shape[:2]
    return cv2.resize(image, (w//2, h//2), interpolation=cv2.INTER_AREA)

def apply_lowpass_filter(image, kernel_size):
    return cv2.GaussianBlur(image, (kernel_size, kernel_size), 0)

def lowpass_frequency_filter(image, cutoff_ratio=0.1):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    rows, cols = gray.shape
    crow, ccol = rows//2, cols//2

    # Create circular mask
    cutoff = int(min(rows, cols) * cutoff_ratio)
    mask = np.zeros((rows, cols), np.uint8)
    cv2.circle(mask, (ccol, crow), cutoff, 1, -1)

    # Frequency domain operations
    dft = np.fft.fft2(gray)
    dft_shift = np.fft.fftshift(dft)
    filtered = dft_shift * mask
    img_back = np.abs(np.fft.ifft2(np.fft.ifftshift(filtered)))

    # Normalize to 0-255
    return cv2.normalize(img_back, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

# Load image with error checking
image_path = '/mnt/data/image.png'
image = cv2.imread(image_path)
if image is None:
    raise FileNotFoundError(f"Image not found at {image_path}")

# Process all operations
results = {
    'Gamma Correction': gamma_correction(image, 2.2),
    'Histogram Equalization': histogram_equalization(image),
    '6-bit': reduce_bit_depth(image, 6),
    '4-bit': reduce_bit_depth(image, 4),
    '2-bit': reduce_bit_depth(image, 2),
    '1-bit': reduce_bit_depth(image, 1),
    'Half Resolution': reduce_spatial_resolution(image),
    '3x3 Filter': apply_lowpass_filter(image, 3),
    '5x5 Filter': apply_lowpass_filter(image, 5),
    '7x7 Filter': apply_lowpass_filter(image, 7),
    'Frequency Filter': lowpass_frequency_filter(image, 0.1)
}

# Display results
titles = list(results.keys())
images = [cv2.cvtColor(img, cv2.COLOR_BGR2RGB) if len(img.shape)==3 else img
          for img in results.values()]

plt.figure(figsize=(18, 12))
for i, (title, img) in enumerate(zip(titles, images)):
    plt.subplot(4, 3, i+1)
    plt.imshow(img, cmap='gray' if title=='Frequency Filter' else None)
    plt.title(title)
    plt.axis('off')

plt.tight_layout()
plt.show()

FileNotFoundError: Image not found at /mnt/data/image.png